In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from os import *
from sklearn.utils import class_weight
import shutil
import random

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [ ]:
def copy_to_dir(data):
    p = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/'
    
    if data == test: 
        p_data = 'Test'
        
    if data == train: 
        p_data = 'Train'
        
    if data == validation: 
        p_data = 'Validation'

    new_path_adr = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/DATA_SETS/{0}/ADR/'.format(p_data)  
    new_path_hrh = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/DATA_SETS/{0}/HRH/'.format(p_data)   
    new_path_dmso = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/DATA_SETS/{0}/DMSO/'.format(p_data)   


    pa_adr = p + 'ADR_cropped//'
    pa_hrh = p + 'HRH_cropped//'
    pa_dmso = p + 'DMSO_cropped/'


    for i in tqdm.tqdm(data):
        
        if 'adr' in i:
            shutil.copy(pa_adr + i, new_path_adr + i)
            
        if 'hrh' in i:
            shutil.copy(pa_hrh + i, new_path_hrh + i)
            
        if 'dmso' in i:
            shutil.copy(pa_dmso + i, new_path_dmso + i)

    
    return 'Done'


In [ ]:
def loadImages(path_data):
    
    p = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/'
    
    
    
    pa_adr = p + 'ADR_tile/'
    pa_hrh = p + 'HRH_tile/'
    pa_dmso = p + 'DMSO_tile/'
    
    image_list = []
    
    
       


    for filename in path_data: 
        
        if 'adr' in filename:
            
            im=cv2.imread(pa_adr + filename,1)

            imarray = np.array(im)
            imarray = imarray.astype('float32')
            

            image_list.append(imarray)
        
            
        if 'hrh' in filename:
            
            im=cv2.imread(pa_hrh + filename,1)

            imarray = np.array(im)
            imarray = imarray.astype('float32')
            

            image_list.append(imarray)
            
        if 'dmso' in filename:
            
            im=cv2.imread(pa_dmso + filename,1)

            imarray = np.array(im)
            imarray = imarray.astype('float32')
            

            image_list.append(imarray)



    x_orig = np.reshape(image_list, (len(image_list), 256, 256, 3))

    return x_orig

In [ ]:
def resize(x):
    rescaled = []

    for i in x:

        scale_percent = 140 # percent of original size
        width = int(i.shape[1] / (scale_percent / 100))
        height = int(i.shape[0] / (scale_percent / 100))
        dim = (width, height)
        resized = cv2.resize(i, dim, interpolation = cv2.INTER_LANCZOS4)
        rescaled.append(resized)

    x_orig = np.reshape( rescaled, (len( rescaled), resized.shape[1], resized.shape[1], 3))

    return x_orig

In [ ]:
def label(y):
    lab = []
    for i in y:
        if 'adr' in i:
            lab.append(0)
        if 'hrh' in i:
            lab.append(1)
        if 'dmso' in i:
            lab.append(2)
    return lab

In [ ]:
met = ['F10']
mid = ['D5']
oxy = ['F6']




cycl = ['C4']
dime =  ['F7']
cypr  = ['G9']


In [ ]:
tot_well_adr = [met,mid,oxy]

tot_well_hrh = [cycl, dime, cypr]

string_well_adr = ['met', 'mid', 'oxy']

string_well_hrh = ['cycl', 'dime', 'cypr']


In [ ]:
tot_well = []
string_well = [] 

In [ ]:
a = 'ADR' # FOR TEST SET
b = 'HRH' # FOR REST
c = 'DMSO'

if a == 'HRH':
    tot_well = tot_well_hrh
    string_well = string_well_hrh
    
if a == 'ADR':
    tot_well = tot_well_adr
    string_well = string_well_adr

In [ ]:
tot_results_accuracy = []

results_lstm = []

In [ ]:
time_points = list(map(str, range(1,34)))

new_time = []
for i in time_points:
    r = '_' + i + '.'
    new_time.append(r)



path_test = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/{}_tile/'.format(a)

# NAME OF THE WELLS CORRESPONDING TO THE DRUG THAT YOU WANT IN THE TEST SET 

wells_drug = [tot_well[0][0]] 

test = []

for _,_, filenames in os.walk(path_test):

    for filename in sorted(filenames, key = natural_keys):

        for w in wells_drug:
            for t in new_time:
                if '{}'.format(w) in filename and '{}tiff'.format(t) in filename:
                    test.append(filename)

groups_list = ['{}'.format(a), '{}'.format(b), '{}'.format(c)]

fileds_of_view = ['1','2','3','4','5']

field_train, field_val = train_test_split(fileds_of_view, test_size=0.4, random_state=5)


train = []

validation = []

group_compounds = []

for group in tqdm.tqdm(groups_list):

    pa = '/home/jovyan/DATA_MASTER_PROJECT/IMG_A549_high_con/{}_tile/'.format(group)

    for _,_, filenames in os.walk(pa):

        for filename in sorted(filenames, key = natural_keys):

            for t in new_time:

                if '_{}-'.format(wells_drug[0]) not in filename  and '{}tiff'.format(t) in filename:

                    group_compounds.append(filename)






for i in group_compounds:

    for f in field_train:
        if '-{}_'.format(f) in i:
            train.append(i)


    for v in field_val:
        if '-{}_'.format(v) in i:
            validation.append(i)


In [ ]:
len(train), len(validation), len(test)

In [ ]:
def image_generator(t, b_size):
    while True:
        samp = np.random.choice(np.arange(0, len(t)), size=b_size, replace=False)
        batch_paths_in =  [t[index] for index in samp]
        batch_output = []
        for i in batch_paths_in:
            batch_output += [i]
        
        batch_x = loadImages(batch_output)
#         batch_x = resize(batch_x)
        batch_x = preprocess_input(batch_x)
        batch_y = label(batch_output)
        batch_y = np.array(list(batch_y))
        batch_y = keras.utils.to_categorical(batch_y, num_classes = 3)
        
        yield batch_x, batch_y
        

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=3)

pretrained_model = VGG16(weights='imagenet',include_top=False, input_shape=(256, 256, 3))

base_model = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('block3_pool').output)

print('Model_loaded')


In [ ]:
w_train = label(train)
weights = class_weight.compute_class_weight('balanced', np.unique(w_train),w_train)
weights = dict(enumerate(weights))

In [ ]:
m4 = Sequential()
m4.add(base_model)


m4.add(BatchNormalization())
m4.add(GlobalAveragePooling2D())
m4.add(Dense(128, activation='relu'))
m4.add(BatchNormalization())
m4.add(Dense(64, activation='relu'))
m4.add(BatchNormalization())
m4.add(Activation('relu'))
m4.add(Dense(3,activation='softmax'))


base_model.trainable = False

opt = keras.optimizers.Adam(lr=1e-3)

m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])


In [ ]:
epochs = 300
batch_size = 256

m4_h = m4.fit(image_generator(train, batch_size),
                steps_per_epoch=len(train)//batch_size,
                callbacks = [es],
                epochs=epochs,
                validation_data = image_generator(validation,batch_size), 
                validation_steps = len(validation)//batch_size,
                class_weight = weights,
                 verbose = 1)